In [24]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv(r"C:\Users\ellah\OneDrive\Data Science Projects\Drug Data\Copy of Accidental_Drug_Related_Deaths_2012-2022.csv")

In [4]:
df.head

<bound method NDFrame.head of              Date      Date Type   Age     Sex                       Race  \
0      05/29/2012  Date of death  37.0    Male                      Black   
1      06/27/2012  Date of death  37.0    Male                      White   
2      03/24/2014  Date of death  28.0    Male                      White   
3      12/31/2014  Date of death  26.0  Female                      White   
4      01/16/2016  Date of death  41.0    Male                      White   
...           ...            ...   ...     ...                        ...   
10649  08/15/2022  Date of death  60.0    Male                      White   
10650  01/30/2022  Date of death  65.0    Male                      White   
10651  09/17/2022  Date of death  41.0    Male  Black or African American   
10652  12/20/2022  Date of death  59.0    Male                      White   
10653  09/08/2022  Date of death  49.0    Male                      White   

      Ethnicity Residence City Residence Coun

In [7]:
df["Date"]=pd.to_datetime(df["Date"])

df["Year"]=df["Date"].dt.year
df["Month"]=df["Date"].dt.month


df = df[df["Date Type"] != "Date reported"]


df["Ethnicity"]= df["Ethnicity"].fillna("Unknown")

ethnicity_map = {
    "Unknown":0,
    "Not Spanish/Hispanic/Latino":0
}
df["Ethnicity"] = df["Ethnicity"].map(ethnicity_map).fillna(1).astype(int)

columns_to_fill = ["Sex", "Description of Injury", "Race", "Residence City", "Residence County", "Residence State", "Injury City", "Injury County", "Injury State", "Injury Place", "Death City", "Death County", "Death State", "Location", "Location if Other",'Cause of Death', 'Manner of Death','Other Significant Conditions ', 'Year', 'Month']
df[columns_to_fill] = df[columns_to_fill].fillna("Unknown")

columns_mean = ["Age"]
df[columns_mean] = df[columns_mean].fillna(df[columns_mean].mean())


columns_to_transform = ["Heroin", 'Cocaine', 'Fentanyl','Fentanyl Analogue', 'Oxycodone', 'Oxymorphone', 'Ethanol',
       'Hydrocodone', 'Benzodiazepine', 'Methadone', 'Meth/Amphetamine',
       'Amphet', 'Tramad', 'Hydromorphone', 'Morphine (Not Heroin)',
       'Xylazine', 'Gabapentin', 'Opiate NOS', 'Heroin/Morph/Codeine',
       'Other Opioid', 'Any Opioid']
df[columns_to_transform] = df[columns_to_transform].replace({"y":1, "Y":1, "Y POPS":1, "Y (PTCH)":1, "P":1, "PCP NEG":0, "Buprenorphine":1,"PCP":1, "MDMA, Buprenorphine":1, "ketamine":1, "Mitragynine":1, "Difluro":1, "pcp":1, "N":0 }).fillna(0).astype(float)


df["Race"] = df["Race"].replace({"Black or African American":"Black"})
df["Race"] = df["Race"].replace({"American Indian or Alaska Native":"Other", "Native American, Other":"Other", "Hawaiian":"Other"})
df["Race"] = df["Race"].replace({"Asian, Other":"Asian", "Other Asian":"Asian", "Chinese":"Asian", "Korean":"Asian", "Asian/Indian":"Asian", "Asian Indian":"Asian"})

df["Combined Opiod"] = ((df['Other Opioid']==1) | (df['Any Opioid']==1) | (df['Opiate NOS']==1)).astype(int)


def my_clip(x):
    q1, q3 = x.quantile([.25,.75])
    iqr = q3 - q1
    low = q1 -(1.5*iqr)
    high = q3 + (1.5*iqr)

    return np.clip(x, low, high)
df["Age"] = pd.to_numeric(df["Age"])
df["Age"] = df["Age"].fillna(df["Age"].mean())
df["Age"] = my_clip(df["Age"])

min_count = 10
df = df[df.apply(lambda x: x.nunique() >= min_count, axis=1)]

df.set_index("Year")

C:\Users\ellah\AppData\Local\Temp\ipykernel_11568\3577185169.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Date"]=pd.to_datetime(df["Date"])
C:\Users\ellah\AppData\Local\Temp\ipykernel_11568\3577185169.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Year"]=df["Date"].dt.year
C:\Users\ellah\AppData\Local\Temp\ipykernel_11568\3577185169.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

,Date,Date Type,Age,Sex,Race,Ethnicity,Residence City,Residence County,Residence State,Injury City,...,Opiate NOS,Heroin/Morph/Codeine,Other Opioid,Any Opioid,Other,ResidenceCityGeo,InjuryCityGeo,DeathCityGeo,Month,Combined Opiod
Year,,,,,,,,,,,,,,,,,,,,,
2012,2012-05-29,Date of death,37.0,Male,Black,1,STAMFORD,FAIRFIELD,Unknown,STAMFORD,...,0.0,0.0,0.0,0.0,NaN,"STAMFORD, CT\n(41.051924, -73.539475)","STAMFORD, CT\n(41.051924, -73.539475)","CT\n(41.575155, -72.738288)",5,0
2012,2012-06-27,Date of death,37.0,Male,White,1,NORWICH,NEW LONDON,Unknown,NORWICH,...,0.0,0.0,0.0,0.0,NaN,"NORWICH, CT\n(41.524304, -72.075821)","NORWICH, CT\n(41.524304, -72.075821)","Norwich, CT\n(41.524304, -72.075821)",6,0
2014,2014-03-24,Date of death,28.0,Male,White,1,HEBRON,Unknown,Unknown,HEBRON,...,0.0,0.0,0.0,0.0,NaN,"HEBRON, CT\n(41.658069, -72.366324)","HEBRON, CT\n(41.658069, -72.366324)","Marlborough, CT\n(41.632043, -72.461309)",3,0
2014,2014-12-31,Date of death,26.0,Female,White,1,BALTIC,Unknown,Unknown,Unknown,...,0.0,0.0,0.0,0.0,NaN,"BALTIC, CT\n(41.617221, -72.085031)","CT\n(41.575155, -72.738288)","Baltic, CT\n(41.617221, -72.085031)",12,0
2016,2016-01-16,Date of death,41.0,Male,White,1,SHELTON,FAIRFIELD,CT,SHELTON,...,0.0,0.0,0.0,1.0,NaN,"SHELTON, CT\n(41.316843, -73.092968)","SHELTON, CT\n(41.316843, -73.092968)","Bridgeport, CT\n(41.179195, -73.189476)",1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022,2022-08-15,Date of death,60.0,Male,White,1,SOUTHINGTON,HARTFORD,CT,SOUTHINGTON,...,0.0,0.0,0.0,1.0,NaN,"SOUTHINGTON, CT\n(41.600220000000036, -72.8781...","SOUTHINGTON, CT\n(41.600220000000036, -72.8781...","CT\n(41.57350273000003, -72.73830590799997)",8,1
2022,2022-01-30,Date of death,65.0,Male,White,1,HARTFORD,HARTFORD,CT,HARTFORD,...,0.0,1.0,1.0,1.0,Buprenorphine,"HARTFORD, CT\n(41.76376000000005, -72.67397999...","HARTFORD, CT\n(41.76376000000005, -72.67397999...","CT\n(41.57350273000003, -72.73830590799997)",1,1
2022,2022-09-17,Date of death,41.0,Male,Black,1,HARTFORD,HARTFORD,CT,HARTFORD,...,0.0,0.0,0.0,1.0,NaN,"HARTFORD, CT\n(41.76376000000005, -72.67397999...","HARTFORD, CT\n(41.76376000000005, -72.67397999...","CT\n(41.57350273000003, -72.73830590799997)",9,1


In [8]:
df.columns

Index(['Date', 'Date Type', 'Age', 'Sex', 'Race', 'Ethnicity',
       'Residence City', 'Residence County', 'Residence State', 'Injury City',
       'Injury County', 'Injury State', 'Injury Place',
       'Description of Injury', 'Death City', 'Death County', 'Death State',
       'Location', 'Location if Other', 'Cause of Death', 'Manner of Death',
       'Other Significant Conditions ', 'Heroin',
       'Heroin death certificate (DC)', 'Cocaine', 'Fentanyl',
       'Fentanyl Analogue', 'Oxycodone', 'Oxymorphone', 'Ethanol',
       'Hydrocodone', 'Benzodiazepine', 'Methadone', 'Meth/Amphetamine',
       'Amphet', 'Tramad', 'Hydromorphone', 'Morphine (Not Heroin)',
       'Xylazine', 'Gabapentin', 'Opiate NOS', 'Heroin/Morph/Codeine',
       'Other Opioid', 'Any Opioid', 'Other', 'ResidenceCityGeo',
       'InjuryCityGeo', 'DeathCityGeo', 'Year', 'Month', 'Combined Opiod'],
      dtype='object')

In [16]:
df["Manner of Death"].nunique()

5

In [17]:
df = df.drop(columns=["Date Type", "Location if Other", "Manner of Death", "ResidenceCityGeo", "InjuryCityGeo", "DeathCityGeo"])

In [19]:
df.shape

(8887, 45)

In [26]:
target_column  = ["Fentanyl"]

y = df[target_column]
X = df.drop(columns=target_column)

In [33]:
df.dtypes.value_counts()

float64           22
object            18
int64              2
int32              2
datetime64[ns]     1
Name: count, dtype: int64

In [35]:
numerical_features = X.select_dtypes(include = ['float64', 'int64', 'int32'])

categorical_features = X.select_dtypes(include = ['object'])

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [37]:
X_train.shape

(7109, 44)

In [38]:
y_train.shape

(7109, 1)